In [47]:
export nnUNet_raw_data_base="/data/nnUNet_results/nnUNet_raw_data_base"
export nnUNet_preprocessed="/data/nnUNet_results/nnUNet_preprocessed"
export RESULTS_FOLDER="/data/nnUNet_results/nnUNet_trained_models"

CUDA_VISIBLE_DEVICES=3 nnUNet_train 2d nnUNetTrainerV2Josh 6 0

In [7]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [9]:
# check whether GPU accelerated computing is available
assert torch.cuda.is_available() # if there is an error here, enable GPU in the Runtime

In [10]:
# check if nnunet can be imported
import nnunet



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [11]:
import wandb
wandb.init(project="nnU-Net_Workshop")
# ae32c1724511b998ba55b2fe6c8cdc16553ea2ae

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [13]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw_data_base" : os.path.join("nnUNet_results", "nnUNet_raw_data_base"), 
    "nnUNet_preprocessed" : os.path.join("nnUNet_results", "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    # "nnUNet_preprocessed" : os.path.join(base_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 108s -> seems faster take this
    "RESULTS_FOLDER" : os.path.join("nnUNet_results", "nnUNet_Results_Folder"),
    "RAW_DATA_PATH" : os.path.join("nnUNet_results", "RawData"), # This is used here only for convenience (not necessary for nnU-Net)!
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path 

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
    if os.getenv(env_var) != path:
        print("Error:")
        print("Environment Variable {} is not set correctly!".format(env_var))
        print("Should be {}".format(path))
        print("Variable is {}".format(os.getenv(env_var)))
    make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /data
nnUNet_results/nnUNet_raw_data_base exists.
nnUNet_results/nnUNet_preprocessed exists.
nnUNet_results/nnUNet_Results_Folder created!
nnUNet_results/RawData created!
If No Error Occured Continue Forward. =)


In [26]:
from nnunet.dataset_conversion.utils import generate_dataset_json
import os 

target_base = "nnUNet_results/nnUNet_raw_data_base/nnUNet_raw_data/Task006_Lung"
target_imagesTr = target_base+ "/imagesTr"
target_imagesTs = target_base+ "/imagesTs"
target_labelsTs = target_base+ "/labelsTs"
target_labelsTr = target_base+ "/labelsTr"

# each extension - HGG or LGG
counter = 0
# each folder in extension
print(target_imagesTr)
path = target_imagesTr
d = []
for files in os.scandir(path):
    if files.is_dir() or files.is_file():
        if not files.name.startswith("."):
            d.append({"image":"./imagesTr/" + files.name,"label":"./labelTr/" + files.name})
counter = len(d)
print(counter,d)

# #dictionary["training"] = d

# import json

# jsonFile = open(target_base + '/dataset.json')
# data = json.load(jsonFile)

# #function to add to JSON
# def write_json(new_data, filename=target_base + '/dataset.json'):
#     with open(filename,'r+') as file:
#           # First we load existing data into a dict.
#         file_data = json.load(file)
#         # Join new_data with file_data inside emp_details
#         file_data["training"].append(new_data)
#         # Sets file's current position at offset.
#         file.seek(0)
#         # convert back to json.
#         json.dump(file_data, file, indent = 4)

# write_json(d)
# # # finally we can call the utility for generating a dataset.json
# # generate_dataset_json(target_base + "/dataset.json", target_imagesTr, target_imagesTs, "0",
# #                           labels={0: 'background', 1: 'Tumour'}, dataset_name="Task06_Lung", license='hands off!')

nnUNet_results/nnUNet_raw_data_base/nnUNet_raw_data/Task006_Lung/imagesTr
63 [{'image': './imagesTr/lung_047.nii.gz', 'label': './labelTr/lung_047.nii.gz'}, {'image': './imagesTr/lung_046.nii.gz', 'label': './labelTr/lung_046.nii.gz'}, {'image': './imagesTr/lung_070.nii.gz', 'label': './labelTr/lung_070.nii.gz'}, {'image': './imagesTr/lung_016.nii.gz', 'label': './labelTr/lung_016.nii.gz'}, {'image': './imagesTr/lung_061.nii.gz', 'label': './labelTr/lung_061.nii.gz'}, {'image': './imagesTr/lung_093.nii.gz', 'label': './labelTr/lung_093.nii.gz'}, {'image': './imagesTr/lung_086.nii.gz', 'label': './labelTr/lung_086.nii.gz'}, {'image': './imagesTr/lung_025.nii.gz', 'label': './labelTr/lung_025.nii.gz'}, {'image': './imagesTr/lung_065.nii.gz', 'label': './labelTr/lung_065.nii.gz'}, {'image': './imagesTr/lung_071.nii.gz', 'label': './labelTr/lung_071.nii.gz'}, {'image': './imagesTr/lung_041.nii.gz', 'label': './labelTr/lung_041.nii.gz'}, {'image': './imagesTr/lung_022.nii.gz', 'label': './l

In [44]:
from nnunet.dataset_conversion.utils import generate_dataset_json
import os 

target_base = "nnUNet_results/nnUNet_raw_data_base/nnUNet_raw_data/Task006_Lung"
target_imagesTr = target_base+ "/imagesTr"
target_imagesTs = target_base+ "/imagesTs"
target_labelsTs = target_base+ "/labelsTs"
target_labelsTr = target_base+ "/labelsTr"

# each extension - HGG or LGG
counter = 0
# each folder in extension
print(target_imagesTs)
path = target_imagesTs

d = []
for files in os.scandir(path):
    if files.is_dir() or files.is_file():
        if not files.name.startswith("."):
            if files.name[8:] != "_0000.nii.gz":
                rename = files.name[:-7] + "_0000.nii.gz"
                os.rename(path + "/" + files.name, path + "/" + files.name[:-7] + "_0000.nii.gz")
            print(rename)
            
            #d.append({"image":"./imagesTr/" + files.name,"label":"./labelTr/" + files.name})
counter = len(d)
print(counter,d)

nnUNet_results/nnUNet_raw_data_base/nnUNet_raw_data/Task006_Lung/imagesTs
lung_060_0000.nii.gz
lung_076_0000.nii.gz
lung_087_0000.nii.gz
lung_039_0000.nii.gz
lung_040_0000.nii.gz
lung_068_0000.nii.gz
lung_019_0000.nii.gz
lung_024_0000.nii.gz
lung_050_0000.nii.gz
lung_011_0000.nii.gz
lung_032_0000.nii.gz
lung_013_0000.nii.gz
lung_035_0000.nii.gz
lung_089_0000.nii.gz
lung_067_0000.nii.gz
lung_090_0000.nii.gz
lung_063_0000.nii.gz
lung_052_0000.nii.gz
lung_007_0000.nii.gz
lung_082_0000.nii.gz
lung_012_0000.nii.gz
lung_030_0000.nii.gz
lung_002_0000.nii.gz
lung_017_0000.nii.gz
lung_077_0000.nii.gz
lung_021_0000.nii.gz
lung_008_0000.nii.gz
lung_091_0000.nii.gz
lung_085_0000.nii.gz
lung_072_0000.nii.gz
lung_088_0000.nii.gz
lung_056_0000.nii.gz
0 []


In [48]:
# Prepare the Execution of nnU-Net for Task 4 - this is the Hippocampus Dataset here (taking 1-2 minutes)
!nnUNet_plan_and_preprocess -t 6



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
RESULTS_FOLDER is not defined and nnU-Net cannot be used for training or inference. If this is not in

In [ ]:
# network can only be one of the following: 
# '2d', 
# '3d_lowres', 
# '3d_fullres', 
# '3d_cascade_fullres'


# nnUNet_train 2d nnUNet_encoder_trainer_josh 6 0


In [10]:
import nibabel as nib
def dataread(path):
    d = []
    
    for (dir_path, dir_names, file_names) in walk(path):
        #print(file_names)
        # gets rid of any pesky leftover .ipynb_checkpoints files
        if not file_names == []:
            for name in range(len(file_names)):
                if not file_names[name].startswith("."):
                    d.append(file_names[name])
    return d

path1 = "nnUNet_results/nnUNet_preprocessed/Task006_Lung/gt_segmentations_/lung_041.nii.gz" 
path2 = "nnUNet_results/nnUNet_preprocessed/Task006_Lung/gt_segmentations/lung_041.nii.gz"

data_Plot = nib.load(path1)
input_2 = data_Plot.get_fdata()

print(input_2.shape)

data_Plot = nib.load(path2)
input_2 = data_Plot.get_fdata()

print(input_2.shape)

(512, 512, 240)
(512, 512, 240)
